In [ ]:
import pandas as pd
import numpy as np
import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Comments Data.csv')

In [ ]:
df.shape

(16461, 12)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16461 entries, 0 to 16460
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   created_time        16461 non-null  object
 1   commentDate         16461 non-null  object
 2   commentAuthor       16461 non-null  object
 3   permalink_url       16461 non-null  object
 4   commentText         16460 non-null  object
 5   feed_source         16461 non-null  object
 6   parentPostId        16461 non-null  object
 7   message             16447 non-null  object
 8   commentId           16461 non-null  object
 9   commentCategory     11447 non-null  object
 10  sentimap.Sentiment  10190 non-null  object
 11  commentAuthorName   10682 non-null  object
dtypes: object(12)
memory usage: 1.5+ MB


In [ ]:
df.head()

,created_time,commentDate,commentAuthor,permalink_url,commentText,feed_source,parentPostId,message,commentId,commentCategory,sentimap.Sentiment,commentAuthorName
0,"[""2022-07-26T05:44:02.000Z"",""2022-07-26T05:44:...",2022-07-26T05:48:34.000Z,17940568721235593,https://www.instagram.com/tv/CgdtJE3vbaV/,Jay hind 🇮🇳,Instagram,17916256646481655,Your daily dose of positivity delivered! #MyGo...,17940568721235593179162566464816551658814514000,Suggestion,NaN,_shivani__gautam_198922
1,"[""2022-07-26T05:44:02.000Z"",""2022-07-26T05:44:...",2022-07-26T05:44:28.000Z,17952781612998552,https://www.instagram.com/tv/CgdtJE3vbaV/,💲𝟏𝟐𝟎𝐫𝐬:- 𝟏𝟎𝟎𝟎 𝐟𝐨𝐥𝐥𝐨𝐰𝐞𝐫𝐬\n💲 𝟐𝟏𝟎𝐫𝐬: 𝟐𝟎𝟎𝟎 𝐟𝐨𝐥𝐥𝐨𝐰𝐞...,Instagram,17916256646481655,Your daily dose of positivity delivered! #MyGo...,17952781612998552179162566464816551658814268000,NaN,NaN,instant__booster_
2,"[""2022-07-26T05:44:02.000Z"",""2022-07-26T05:44:...",2022-07-26T05:45:50.000Z,17897535698640852,https://www.instagram.com/tv/CgdtJE3vbaV/,Power of democracy of India 💪🎊🎉🥳🇮🇳,Instagram,17916256646481655,Your daily dose of positivity delivered! #MyGo...,17897535698640852179162566464816551658814350000,Suggestion,NaN,maaprajitasportsvp
3,"[""2022-07-26T05:44:02.000Z"",""2022-07-26T05:44:...",2022-07-26T06:01:47.000Z,17935734572489124,https://www.instagram.com/tv/CgdtJE3vbaV/,🔥👏,Instagram,17916256646481655,Your daily dose of positivity delivered! #MyGo...,17935734572489124179162566464816551658815307000,NaN,NaN,_tyagiishubham_
4,"[""2022-07-26T05:44:02.000Z"",""2022-07-26T05:44:...",2022-07-26T06:14:34.000Z,17930411201249794,https://www.instagram.com/tv/CgdtJE3vbaV/,*Bhaarath maata ke jay... *भारत माता की जय... ...,Instagram,17916256646481655,Your daily dose of positivity delivered! #MyGo...,17930411201249794179162566464816551658816074000,spam,NaN,_y5_candy_boy_


In [ ]:
 
df = df.drop("commentDate", axis=1) 
df = df.drop("commentAuthor", axis=1) 
df = df.drop("permalink_url", axis=1) 
df = df.drop("feed_source", axis=1) 
df = df.drop("parentPostId", axis=1)
df = df.drop("message", axis=1) 
df = df.drop("commentId", axis=1)
df = df.drop("commentCategory", axis=1)
df = df.drop("commentAuthorName", axis=1)


In [ ]:
df.head()

,commentText,sentimap.Sentiment
0,Jay hind 🇮🇳,NaN
1,💲𝟏𝟐𝟎𝐫𝐬:- 𝟏𝟎𝟎𝟎 𝐟𝐨𝐥𝐥𝐨𝐰𝐞𝐫𝐬\n💲 𝟐𝟏𝟎𝐫𝐬: 𝟐𝟎𝟎𝟎 𝐟𝐨𝐥𝐥𝐨𝐰𝐞...,NaN
2,Power of democracy of India 💪🎊🎉🥳🇮🇳,NaN
3,🔥👏,NaN
4,*Bhaarath maata ke jay... *भारत माता की जय... ...,NaN


In [ ]:
df.isnull().sum()

commentText              1
sentimap.Sentiment    6271
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.isnull().sum()

commentText           0
sentimap.Sentiment    0
dtype: int64

In [ ]:
df.shape

(10189, 2)

In [ ]:
df. rename(columns = {'sentimap.Sentiment':'Sentiment'}, inplace = True)

In [ ]:
df.Sentiment.unique()

array(['Neutral', 'Negitive', 'Positive'], dtype=object)

In [ ]:
df.Sentiment.value_counts()

Neutral     4804
Positive    4228
Negitive    1157
Name: Sentiment, dtype: int64

In [ ]:
df.head()

,commentText,Sentiment
6,@mygovindia https://t.co/pbz5RpLjX5,Neutral
12,Jai hind ki senaa,Neutral
13,Jay ho bharat maa ke veer,Neutral
15,Jay hind,Neutral
26,Jay hind🔥,Neutral


In [ ]:
# operate on the texts
nlp = spacy.load("en_core_web_sm")
stopwords=list(STOP_WORDS)
punctuations=string.punctuation

In [ ]:
# tokenizer

def tokenizer(sentence):
  mytokens=nlp(sentence)
  mytokens= [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
  mytokens= [ word for word in mytokens if word not in stopwords and word not in punctuations ]
  return mytokens

In [ ]:
# Transformation (basically cleaning the text)
class predictors(TransformerMixin):
  def transform(self, X, **transform_params):
    return [clean_text(text) for text in X]
  def fit(self, X, y, **fit_params):
    return self
  def get_params(self, deep=True):
    return {}

# Basic function to clean the text
def clean_text(text):
  return text.strip().lower()

In [ ]:
# Vectorization
vectorizer = CountVectorizer(tokenizer=tokenizer, ngram_range=(1,1))
tfvectorizer = TfidfVectorizer(tokenizer=tokenizer)

In [ ]:
# split dataset
X=df['commentText']
y=df['Sentiment']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)

In [ ]:
# pipeline
SVCclassifier=LinearSVC()
SVCmodel=Pipeline([("cleaner", predictors()),
                   ("vectorizer", vectorizer),
                   ("classifier", SVCclassifier)])

In [ ]:
# train
SVCmodel.fit(X_train,y_train)
SVCpred=SVCmodel.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
# evaluate
confusion_matrix(y_test,SVCpred)

array([[116,  64,  46],
       [ 21, 864,  78],
       [ 30, 118, 701]])

In [ ]:
accuracy_score(y_test,SVCpred)

0.8248282630029441

In [ ]:
print(classification_report(y_test,SVCpred))

              precision    recall  f1-score   support

    Negitive       0.69      0.51      0.59       226
     Neutral       0.83      0.90      0.86       963
    Positive       0.85      0.83      0.84       849

    accuracy                           0.82      2038
   macro avg       0.79      0.75      0.76      2038
weighted avg       0.82      0.82      0.82      2038



In [ ]:
pre=SVCmodel.predict(["Modiji is doing amazing work."])
print("Prediction : ",pre[0])

Prediction :  Positive


In [ ]:
pre=SVCmodel.predict(["Mygov didnot react to my comment"])
print("Prediction : ",pre[0])

Prediction :  Neutral


In [ ]:
pre=SVCmodel.predict(["I do not like man ki baat theme this time"])
print("Prediction : ",pre[0])

Prediction :  Neutral


In [ ]:
pre=SVCmodel.predict(["Even I don't understand this. On one side, Government wants to eradicate caste system and clashes and make everyone equal. On the other side, Gov. is promoting the same."])
print("Prediction : ",pre[0])

Prediction :  Negitive


In [ ]:
pre=SVCmodel.predict(["BJP Government is hypocrite. They make false promises"])
print("Prediction : ",pre[0])

Prediction :  Negitive
